In [1]:
import labbox_ephys as le
import labbox_ephys_widgets_jp as lew

h5py version > 2.10.0. Some extractors might not work properly. It is recommended to downgrade to version 2.10.0: 
>>> pip install h5py==2.10.0


In [2]:
# Load the default workspace
workspace = le.load_workspace()

/home/magland/miniconda3/envs/labbox-ephys-widgets-jp/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# View the workspace
lew.WorkspaceView(workspace=workspace)

WorkspaceViewJp()

# Add recordings/sortings to the workspace

Each time you run this cell, new data will be added to the workspace, which will persist in the loaded feed

In [4]:
import spikeextractors as se
import numpy as np

recording_label = 'simulated_recording'
duration_sec = 50 # duration of simulated recording
num_channels = 8 # num. channels in simulated recording
num_units = 5 # num units
seed = 1 # random number generator seed

def prepare_recording_sorting():
    # Simulate a recording (toy example)
    recording, sorting = se.example_datasets.toy_example(duration=duration_sec, num_channels=num_channels, K=num_units, seed=seed)
    R = le.LabboxEphysRecordingExtractor.from_memory(recording, serialize=True, serialize_dtype=np.int16)
    S = le.LabboxEphysSortingExtractor.from_memory(sorting, serialize=True)
    return R, S

recording, sorting_true = prepare_recording_sorting()
sorting_label = 'true'
R_id = workspace.add_recording(recording=recording, label=recording_label)
S_id = workspace.add_sorting(sorting=sorting_true, recording_id=R_id, label=sorting_label)

/home/magland/miniconda3/envs/labbox-ephys-widgets-jp/lib/python3.8/site-packages/spikeextractors/extraction_tools.py:877: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(start_frame, (float, np.float)):
/home/magland/miniconda3/envs/labbox-ephys-widgets-jp/lib/python3.8/site-packages/spikeextractors/extraction_tools.py:883: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devd

Adding recording: R-579ac77e1be4
Adding sorting: S-8ddbbbbb5182
